In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data/'
group_dir_ = os.path.join(dir_, 'groups/high')
folder = 'sp_matrix_pop(new)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

In [2]:
all_user = pd.read_pickle(os.path.join(dir_, folder, file_name[:-3] + 'pkl'))
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
7,0,3,0.25,1
11,0,7,2.50,3
32,0,18,1.50,2
34,0,20,2.50,3
45,0,31,5.00,17


In [3]:
predictions[:5]

,uid,tid,rating
0,0,3,0.099452
1,0,7,0.095271
2,0,18,0.099722
3,0,20,0.090790
4,0,31,0.110214


In [4]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
7,0,3,0.25,1
11,0,7,2.50,3
32,0,18,1.50,2
34,0,20,2.50,3
45,0,31,5.00,17


In [5]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [6]:
num_user = len(predictions['uid'].unique())
num_user

220

In [7]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [8]:
pop_4 = []
pop_20 = []
pop_50 = []

In [9]:
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops =[]
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    if top_n_size == 4:
        pop_4.append(avg_pops)
    elif top_n_size == 20:
        pop_20.append(avg_pops)
    elif top_n_size == 50:
        pop_50.append(avg_pops)

In [10]:
for i in pop_4:
    print(i)
for i in pop_20:
    print(i)
for i in pop_50:
    print(i)

[0.551991242702254, 0.5205018939393954, 0.4832281144781153, 0.4550475687103606, 0.4398674242424236, 0.41726539589442724, 0.4046296296296316, 0.4089015151515155, 0.37954545454545474, 0.3469318181818193]
[0.4758924103419599, 0.4442077020202101, 0.414726430976438, 0.39262684989429336, 0.37952020202019954, 0.3640029325513142, 0.3544023569023479, 0.3485606060606014, 0.3389935064935019, 0.31620454545453974]
[0.42539949958298423, 0.3960303030303082, 0.37074074074075003, 0.35152854122621874, 0.33956060606060573, 0.3300087976539647, 0.32337373737374364, 0.3152348484848512, 0.3133852813852829, 0.2927909090909087]
